In [3]:
import numpy as np
np.random.seed(42)
import torch
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
import random
random.seed(42)

import os
import subprocess
from tqdm import tqdm
from os import makedirs
from gaussian_renderer import render_lie, render_qtvec
import torchvision
from utils.general_utils import safe_state
from argparse import ArgumentParser
from arguments import ModelParams, PipelineParams, get_combined_args, pose_estimation_params
from gaussian_renderer import GaussianModel
from utils.image_utils import psnr
from utils.loss_utils import EdgeDetection
import numpy as np
from utils.loss_utils import l1_loss, ssim, l2_loss
from copy import deepcopy
from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity as LPIPS
import pypose as pp

from scene.dataset_readers import readColmapSceneInfo
from utils.camera_utils import loadCam
from collections import namedtuple
from matplotlib import pyplot as plt


import torch
import math
from diff_gaussian_rasterization_lie_pp import GaussianRasterizationSettings, GaussianRasterizer
from scene.gaussian_model import GaussianModel
from utils.sh_utils import eval_sh

import pickle

from scene.dataset_readers import CameraInfo
from PIL import Image
import mrob
import open3d as o3d

from arguments import  pose_estimation_params
from utils.general_utils import get_expon_lr_func
import cv2
from glob import glob
import copy
import json

def get_number_of_max_iou_iters(num, max_iters=16):
    return num if num < max_iters else max_iters

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
tasks_root = '/mnt/sdb1/home/kbotashev/iros_paper/replica_dataset_1/results_res4/tasks'
number_of_max_iou_iters = 8

for room in ['apartment_0', 'apartment_1', 'office_0', 'office_2', 'office_4']:
    for camera_type in ['lietorch']:
    # for camera_type in ['lietorch', 'qtvec']:
        room_task_info = {  
                            'room_name':room,
                            # 'camera_type':camera_type,
                            # # 'camera_type':'qtvec',
                            # 'solving_method':'vanilla',
                            # 'solving_method_args': None,
                            # 'loss_type':'l1',
                            # 'optimizer_type':'adam',
                            # 'init_render_resolution':2,
                            # 'iterations':2000,
                            # 'exit_psnr_parameter':1e-4,
                            # 'pose_lr_init':0.01,
                            # 'pose_lr_final':1e-5,
                            # 'pose_lr_delay_steps':0,
                            # 'pose_lr_delay_mult':0,
                            'start_frame':0,
                            'last_frame':32,
                            'experiments_base_dir':'/mnt/sdb1/home/kbotashev/iros_paper/replica_dataset_1/'
                        }

        ply_path = os.path.join(room_task_info['experiments_base_dir'], 
                                'output', room_task_info['room_name'], 
                                'point_cloud/iteration_40000/point_cloud.ply')

        frames_info_pickle_paths = sorted(glob(os.path.join(room_task_info['experiments_base_dir'], 
                                        'scenes', room_task_info['room_name'], 
                                        room_task_info['room_name']+'_task', 'images_pairs_light/*.pickle')))

        frames_info_full = {}
        frames_results_full = copy.copy(room_task_info)
        frames_results_full['results'] = {}

        for frames_info_pickle_path in tqdm(frames_info_pickle_paths[room_task_info['start_frame']:room_task_info['last_frame']]):
            frame_name = frames_info_pickle_path.split('/')[-1].split('.pickle')[0]
            frames_results_full['results'][frame_name] = {}
            frames_results_full['results'][frame_name]['valid_iou_bins_indices'] = {}
            with open(frames_info_pickle_path, 'rb') as handle:
                frames_info_full[frame_name] = pickle.load(handle)
            for iou_key in frames_info_full[frame_name]['frames']['init_frames_iou_bins'].keys():
                frames_results_full['results'][frame_name]['valid_iou_bins_indices'][iou_key] = []
                for idx, frame in enumerate(frames_info_full[frame_name]['frames']['init_frames_iou_bins'][iou_key]):
                    if frame['black_pixels_ratio'] < 0.1:
                        frames_results_full['results'][frame_name]['valid_iou_bins_indices'][iou_key].append(idx)

            frame_iou_stats = frames_info_full[frame_name]['frames']['init_frames_iou_bins_stats']
            iou_bins = [0.05, 0.1, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.50, 0.65]
            # iou_bins = frame_iou_stats.keys()
            # frames_results_full['results'][frame_name]['desired_frame_result_stats'] = {iou:get_number_of_max_iou_iters(frame_iou_stats[iou]) for iou in iou_bins}
            # frames_results_full['results'][frame_name]['current_frame_result_stats'] = {iou:0 for iou in iou_bins}
            # frames_results_full['results'][frame_name]['frame_results'] = {iou:{} for iou in iou_bins}

        for frame_name in frames_results_full['results'].keys():
            stats = frames_results_full['results'][frame_name]['valid_iou_bins_indices']
            for iou in iou_bins:
                for init_id in stats[iou][:number_of_max_iou_iters]:
                    task_info = copy.copy(room_task_info)
                    del task_info['start_frame']
                    del task_info['last_frame']
                    pickle_path = os.path.join(room_task_info['experiments_base_dir'], 
                                        'scenes', room_task_info['room_name'], 
                                        room_task_info['room_name']+'_task', 'images_pairs_light',  frame_name + '.pickle')
                    task_info['ply_path'] = ply_path
                    task_info['pickle_path'] = pickle_path
                    task_info['init_id'] = init_id
                    task_info['iou_bin'] = iou
                    task_info['frame_name'] = frame_name
                    
                    room_task_dir = os.path.join(tasks_root, task_info['room_name'])
                    # experiment_task_dir = os.path.join(room_task_dir, 
                    #                                 str(task_info['camera_type']) + '_' + 
                    #                                 str(task_info['loss_type']) + '_' + 
                    #                                 str(task_info['optimizer_type']) + '_' + 
                    #                                 str(task_info['init_render_resolution']) + '_' + 
                    #                                 str(task_info['iterations']) + '_' + 
                    #                                 str(task_info['pose_lr_init']) + '_' + 
                    #                                 str(task_info['pose_lr_final']) + '_' + 
                    #                                 str(task_info['pose_lr_delay_steps']) + '_' + 
                    #                                 str(task_info['pose_lr_delay_mult']) + '_' + 
                    #                                 str(task_info['exit_psnr_parameter']) + '_' + 
                    #                                 str(task_info['solving_method']) + '_' + 
                    #                                 str(task_info['solving_method_args']))
                    frame_experiment_task_dir = os.path.join(room_task_dir, frame_name, ('%.2f'%iou).replace('.', ''))

                    os.makedirs(frame_experiment_task_dir, exist_ok=True)

                    id_iou_frame_json_name = str(init_id).zfill(3) + '_' + ('%.2f'%iou).replace('.', '') + '_' +  frame_name + '.json'

                    json_path = os.path.join(frame_experiment_task_dir, id_iou_frame_json_name)

                    with open(json_path, 'w') as f:
                        json.dump(task_info, f, indent=4)

  0%|          | 0/32 [00:00<?, ?it/s]

100%|██████████| 32/32 [00:00<00:00, 70.33it/s] 


In [2]:
step_frame = 1
room_names = ['apartment_0', 'office_0', 'office_2', 'apartment_1', 'office_4']
frame_num = np.arange(32)
camera_types = ['lietorch']
solving_methods = ['coarse']
init_render_resolutions = [2]

# step_frame = 1
# room_names = ['apartment_0', 'office_0', 'office_2', 'apartment_1', 'office_4']
# frame_num = np.arange(32)
# camera_types = ['qtvec', 'lietorch']
# solving_methods = ['vanilla', 'coarse']

# loss_types=['l1']
# optimizer_types=['adam']
# init_render_resolutions=[2]
# iterations_params=[2000]
# exit_psnr_parameters=[5e-5],
# pose_lrs_init=[0.01]
# pose_lrs_final=[1e-5]
# pose_lrs_delay_steps=[0]
# pose_lrs_delay_mult=[0]

# solving_methods_args = [{ 
#                         'max_scale':2,
#                         'num_tries':5,
#                         'blur_2d_c2f_kernel_size':201,
#                         'blur_2d_c2f_schedule':[0.05, 0.025, 0.0125, 0.00625, 0.00625, 0.0, 0.0, 0.0, 0.0, 0.0],
#                        }]

NameError: name 'np' is not defined

In [7]:
combinations = []
for start_frame in frame_num:
    for camera_type in camera_types:
            for room_name in room_names:
                for solving_method in solving_methods:
                    for init_render_resolution in init_render_resolutions:
                        combinations.append([room_name, start_frame, start_frame+1, camera_type, solving_method, init_render_resolution])

In [9]:
len(combinations)

320

In [10]:
# combinations += combinations[-16:]
combinations = np.array(combinations).reshape(8,2, 20, 6)

In [14]:
output_bash_dir = '/mnt/sdb1/home/kbotashev/iros_paper/replica_dataset_1/results_res4/input_bashs'
min_gpu = 0
terminal_lines = []
for gpu_id, combo6 in enumerate(combinations):
    for gpu_sub_id, combo3 in enumerate(combo6):
        lines = []
        for combo in combo3:
            lines.append('\nCUDA_VISIBLE_DEVICES='+str(gpu_id+min_gpu) + 
                  ', /mnt/sdb1/home/kbotashev/anaconda3/envs/gs102/bin/python' + 
                  ' /mnt/sdb1/home/kbotashev/mip-nerf_projects/gaussian_splatting_original/gaussian-splatting/solve_pose_estimation.py ' + \
                    '--room_name='+combo[0] + ' --start_frame='+combo[1] + ' --end_frame='+combo[2] + \
                    ' --camera_type='+combo[3] + ' --solving_method='+combo[4] + ' --init_render_resolution='+combo[5])
        f = open(os.path.join(output_bash_dir, str(gpu_id+min_gpu) + '_' + str(gpu_sub_id) + ".sh"), "w")
        f.writelines(lines)
        f.close()
        terminal_lines.append('\nnohup bash ' + os.path.join(output_bash_dir, str(gpu_id+min_gpu) + 
                                                           '_' + str(gpu_sub_id) + ".sh") + ' > ' + 
                                                           os.path.join(output_bash_dir, str(gpu_id+min_gpu) + '_' + str(gpu_sub_id) + '_log.out &'))

f = open(os.path.join(output_bash_dir, "run_bashes.sh"), "w")
f.writelines(terminal_lines)
f.close()

In [92]:
len(glob('/mnt/sdb1/home/kbotashev/iros_paper/replica_dataset_1/results/solutions/qtvec_l1_adam_2_2000_0.01_1e-05_0_0_0.0001_vanilla_None/*/*/*/*_1.pickle'))/len(glob('/mnt/sdb1/home/kbotashev/iros_paper/replica_dataset_1/results/solutions/qtvec_l1_adam_2_2000_0.01_1e-05_0_0_0.0001_vanilla_None/*/*/*/*.pickle'))

0.68408262454435

In [93]:
len(glob('/mnt/sdb1/home/kbotashev/iros_paper/replica_dataset_1/results/solutions/lietorch_l1_adam_2_2000_0.01_1e-05_0_0_0.0001_vanilla_None/*/*/*/*_1.pickle'))/len(glob('/mnt/sdb1/home/kbotashev/iros_paper/replica_dataset_1/results/solutions/lietorch_l1_adam_2_2000_0.01_1e-05_0_0_0.0001_vanilla_None/*/*/*/*.pickle'))

0.42594091460946987

In [26]:
len(glob('/mnt/sdb1/home/kbotashev/iros_paper/replica_dataset/results/tasks/*/*/*/*.json'))

0